<a href="https://colab.research.google.com/github/carlosks/carlosks/blob/main/finetuning_summarizer_Desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-bojeenti/unsloth_a282d091a42449dd96acd8e00be63ab1
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-bojeenti/unsloth_a282d091a42449dd96acd8e00be63ab1
  Resolved https://github.com/unslothai/unsloth.git to commit c730659de7a0a9e0520d01184d5dad2503b52285
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 58.5 MB/s eta 0:00:00
  Created wheel for unsloth: filename=unsloth-2024.9-py3-none-any.whl size=153097 sha256=fe96905c65bc2b80f1646aa3059285ddcdc6eab8757f48dfc458f2f2e732a2df
  Stored in directory: /tmp/pip-ephem-wheel-cache-jtin_u1l/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth
  Attempting unin

In [2]:
!pip install --upgrade accelerate
!pip install --upgrade transformers
!pip install --upgrade peft
!pip install --upgrade bitsandbytes
!pip install --upgrade trl
!pip install --upgrade datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beh

In [6]:
!pip install triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 6.2 MB/s eta 0:00:00


In [10]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

DATA_PATH = "/content/drive/MyDrive/Arquivos_Aula_1_Fine_Tunning/news_dataset_chat_data.json"
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Arquivos_Aula_2_Fine_Tunning/formatted_news_dataset_chat_data.json"
max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]


In [15]:
def format_dataset_into_model_input(data):
    def separate_text(full_text):
        news_start = full_text.find("[|News|]") + len("[|News|]")
        news_end = full_text.find("[|eNews|]")
        summary_start = full_text.find("[|summary|]") + len("[|summary|]")
        summary_end = full_text.find("[|esummary|]")

        instruction = full_text.split('\n')[0]
        input_text = full_text[news_start:news_end].strip()
        response = full_text[summary_start:summary_end].strip()

        return instruction, input_text, response

    # Inicializando as listas para armazenar os dados
    instructions = []
    inputs = []
    outputs = []

    # Check if 'train' is a key in the dictionary and if its value is a list
    if 'train' in data and isinstance(data['train'], list):
        # Accessing the list correctly
        for item in data['train']: # Iterating through the list within 'train'
            prompt = item['input'] # Accessing the 'input' value within each item
            instruction, input_text, response = separate_text(prompt)
            instructions.append(instruction)
            inputs.append(input_text)
            outputs.append(response)
    else:
        print("Error: 'train' key not found or its value is not a list.")

    # Criando o dicionário final
    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    # Salvando o resultado em um arquivo JSON
    with open(OUTPUT_PATH_DATASET, 'w') as output_file:
        json.dump(formatted_data, output_file, indent=4)

    print(f"Dataset salvo em {OUTPUT_PATH_DATASET}")

In [16]:
# Load the dataset
with open(DATA_PATH, 'r') as f:
    data = json.load(f)

format_dataset_into_model_input(data)


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

Error: 'train' key not found or its value is not a list.
Dataset salvo em /content/drive/MyDrive/Arquivos_Aula_2_Fine_Tunning/formatted_news_dataset_chat_data.json
==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [17]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [19]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Arquivos_Aula_2_Fine_Tunning/formatted_news_dataset_chat_data.json"

dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [20]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [21]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1 | Num Epochs = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,0.814900
2,0.814900
3,0.777400
4,0.674700
5,0.547400
6,0.399100
7,0.271500
8,0.153600
9,0.067400
10,0.054800


In [ ]:

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "SUMMARIZE THIS NEWS",
        "Following the death of the Danish butter magnate Lars Emil Bruun in 1923, his will had a curious order: His vast accumulation of coins, notes and medals, amassed over more than six decades, should be held as an emergency reserve for Denmark’s national collection in case it were ever destroyed. After a century, if all was well, his own cache could finally be sold to benefit his descendants. On Tuesday, just under a year since the 100-year-old order expired, the first set of coins from Bruun’s personal 20,000-piece collection went up for auction in Copenhagen. After nearly eight hours of bidding, the opening 286 lots sold for a combined 14.82 million euros ($16.5 million). It will take several more sales to empty Bruun’s coffers, but once completed, it will be the most expensive international coin collection ever sold, according to Stack’s Bowers, the rare coin dealer and auction house hosting the sales. The L.E. Bruun Collection was insured for 500 million Danish kroner, or around $72.5 million. The auction house described it as the most valuable collection of world coins to ever come to market. In a press release following Tuesday’s auction, Stack’s Bowers Galleries’ president, Brian Kendrella, described the first sale as “truly a landmark event for the world coin market.” Where the numismatist’s collection has resided over the past century had been something of a mystery, its location known to few.But Bruun believed that hiding his treasure was for a noble cause; following the destructionhe saw of World War I, he feared the Royal Danish Coin and Medal Collection could one day face bombing or looting, according to the auction house. Bruun began collecting currency as a child in 1859, when his uncle died and named him among the recipients of some of his coins, according to the sales catalog. The son of innkeepers and landowners, he learned in his 20sthat his family inheritance had been squandered and he was saddled in debt. He began his own business in butter with a loan, eventually earning a fortune from sales and exports. With his wealth, he became a prolific coin collector, and was a founding member of the Danish Numismatic Society in 1885. “The good thing about collecting coins is that when you are upset about something or you feel unsettled, then you go and look at your coins, and then you calm down by studying them again and again pondering the many unsolved problems they present,” he once told a Danish magazine, per the catalog. “People who are exclusively devoted to their business make a great mistake. I, for one, could never imagine thinking about nothing but butter until my dying days.” Tuesday’s sale included gold and silver coins from Denmark, Norway and Sweden, dated from the late 15th century to the latter years of Bruun’s life. The star lot was one of Scandinavia’s oldest gold coins, according to the catalog a noble of King Hans dated from 1496. The coin smashed auction house estimates to fetch 1.2 million euros ($1.34 million), setting a new world record for a Scandinavian coin at auction, Stack’s Bowers Galleries said. “Hands-down my favorite piece in the sale is the 1496 gold noble of King Hans, who was king of Denmark and Norway under the Kalmar Union, as well as Sweden for a brief time,” said Matt Orsini, director of world and ancient numismatics at Stack’s Bowers Galleries, in a press statement prior to the first auction. “It is important on so many levels — it’s the very first gold coin struck by Denmark, it’s the very first dated coin struck by the Danish kingdom, and it’s unique in private hands.” Over the past few months, the coins toured different fairs and exhibited at Stack’s Bowers’ galleries. They were also put on display in Copenhagen just before the sale.[|eNews|]\n\n[|summary|]After a century, the first set of coins from Danish butter magnate Lars Emil Bruun's 20,000-piece collection went up for auction in Copenhagen, selling for a combined 14.82 million euros. The collection, insured for $72.5 million, is expected to be.", # input
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "SUMMARIZE THIS NEWS",
        "If home is where the heart is, a new survey suggests that most people aren't sure exactly where they live. More than half of people cannot pinpoint the exact location of the human heart on a diagram, and nearly 70 percent can't correctly identify the shape of the lungs, according to the survey. This lack of knowledge isn't just embarrassing -- it could lead to a poorer quality of health care, some experts say. In the study, published in the journal BMC Family Practice, a research team surveyed 722 Britons -- 589 hospital outpatients and 133 people in the general population. They gave the volunteers four diagrams of human figures and asked them to choose the one that showed the correct size and location of a specific organ. (For example, the heart diagrams showed various size organs on the far left side of the chest, directly in the center, anchored on the center/left chest, and on the right side of the chest.) Overall, people knew less basic anatomy than the researchers expected -- even those patients being treated for a specific condition involving that organ. Participants generally answered half the questions correctly, including 46.5 percent who knew which drawing represented their heart. In all, 31.4 percent correctly identified the lungs, 38.4 percent the stomach, 41.8 percent the thyroid, and 42.5 percent the kidneys. The intestines and bladder were the most easily identified, with 85.9 percent and 80.7 percent, respectively, answering the question correctly. Health.com: Are you cholesterol smart? Take this quiz There was little to no improvement compared with a similar study conducted in 1970, says lead author John Weinman, Ph.D., of King's College London. In that study, subjects correctly identified eight major body parts about half of the time. (The researchers used the same body parts from the 1970 study and added three more: the pancreas, gallbladder, and ovaries.) Given the accessibility of the Internet and the prominence of health stories in the news media today, Weinman's team expected that people would now know more about their body. Weinman says he wouldn't be surprised if a study based in the United States produced similar results -- or worse. \"I imagine they would be similar, but there could well be regional variation, depending on which part of the U.S. the participants were from,\" he says. \"Actually, I asked one of my colleagues, who is from the U.S., and she felt that Americans might be worse because, to quote her, 'Very many Americans don't even know where New Jersey is, so how would they know where their pancreas is?'\" Health.com: Eat Smarter in your 30s, 40s, and 50s That may sound harsh, but time and again, U.S. studies have shown that doctors overestimate how much their patients understand about their conditions and treatment. Adam Kelly, Ph.D., an assistant professor of medicine at the Baylor College of Medicine, in Houston,  Texas, conducted a 2007 study, that showed that doctors overestimate patient literacy and that a lack of patient knowledge leads to poorer care. Kelly believes the problem could be \"even more profound\" in the United States, although a similar study has not been conducted in America. Still, anatomy may not be the best measure of health literacy, says Sandeep Jauhar, M.D., the director of the heart failure program at the Long Island Jewish Medical Center, and the author of \"Intern: A Doctor's Initiation.\" Health.com: Computer games that boost your memory \"They would like us to draw the conclusion that because the patients can't identify these organs anatomically that that is an indication of low health-care literacy -- and that may or may not be true,\" he says. \"I work with heart failure patients, and whether they can identify where their heart is is not so important to me as long as they know which medicines to take and when.\" Many patients with heart failure, unfortunately, don't know which medicines to take, can't identify their symptoms, and don't follow up with their doctors, Jauhar says.", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

In [23]:
model.save_pretrained("/content/drive/MyDrive/Arquivos_Aula_2_Fine_Tunning/lora_model") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/Arquivos_Aula_2_Fine_Tunning/lora_model")


('/content/drive/MyDrive/Arquivos_Aula_2_Fine_Tunning/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/Arquivos_Aula_2_Fine_Tunning/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/Arquivos_Aula_2_Fine_Tunning/lora_model/tokenizer.json')

In [26]:
import json
news_test = []

with open('/content/drive/MyDrive/Arquivos_Aula_2_Fine_Tunning/test_news/news_contents.json', 'r') as news_file:
  news_test = json.loads(news_file.read())['news_content']

In [27]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/content/drive/MyDrive/Arquivos_Aula_2_Fine_Tunning/lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)



==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [28]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


inputs = tokenizer(
[
alpaca_prompt.format(
        "SUMMARIZE THIS NEWS",
        news_test[1],
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
SUMMARIZE THIS NEWS

### Input:
Award-winning Nigerian singer Adekunle Gold rose to international fame when his debut album “Gold” reached number seven on the Billboard World Charts. Since then, the 37-year-old has released four more albums, amassing hundreds of millions of streams worldwide. Behind the scenes, however, he has been quietly battlingsickle cell disease, an inherited blood disorder that leads to abnormally shaped red blood cells, causing severe pain, anemia, and potentially life-threatening complications. It affects children who inherit two copies of the sickle cell gene, one from each parent. The disease is most prevalent in Africa, which accounts for66% of cases worldwide,according to the World Health Organization. Nigeria bears the highest burden, whereas many as 150,000 babiesar